In [90]:
# To start mysql docker: sudo docker-compose up -d

import datajoint as dj
import os
import sys
import json


In [91]:
root_path = os.path.abspath(os.path.join(globals()['_dh'][0], '../..'))

In [3]:
local_cred_filename = os.path.join(root_path, 'datajoint/local_cred.ini')
with open(local_cred_filename) as f:
    local_cred = json.load(f)

In [4]:
local_cred

In [5]:
cred_filename = "~/"

dj.config['database.host'] = local_cred['host']
dj.config['database.user'] = local_cred['user']
dj.config['database.password'] = local_cred['password']

In [10]:
schema = dj.schema('test', locals())

In [7]:
schema.drop()

In [74]:
parent.drop()
child.drop()

In [75]:
# Testing multiple  

@schema
class Parent(dj.Manual):
    definition = """
    parent_id: int
    ---
    parent_data = NULL: varchar(20)
    """

@schema
class Middle(dj.Manual):
    definition = """
    -> Parent
    middle_id: int
    ---
    middle_data: varchar(20)
    """
    
@schema
class Child(dj.Manual):
    definition = """
    child_id: int
    ---
    child_data: varchar(20)
    """
    
    class ParentSrc(dj.Part):
        definition = """
        -> Middle
        -> Child
        """

In [76]:
parent = Parent()
middle = Middle()
child = Child()

In [77]:
parent.insert([{'parent_id': 0, 'parent_data': 'stuff0'}])
parent.insert1({'parent_id': 1})
parent.insert([{'parent_id': 2}])
parent.insert1({'parent_id': 3, 'parent_data': 'stuff3'})
parent.insert1({'parent_id': 10, 'parent_data': 'stuff10'})
parent

In [85]:
p_id, p_data = parent.fetch('parent_id', 'parent_data')


In [86]:
middle.insert([{'parent_id': 0, 'middle_id': 0, 'middle_data': 'mid0'},
               {'parent_id': 1, 'middle_id': 1, 'middle_data': 'mid1'}])
middle

In [87]:
child.insert([{'child_id': 0, 'child_data': 'more0'},
              {'child_id': 1, 'child_data': 'more1'}])
child

In [88]:
child.ParentSrc.insert([{'parent_id': 0, 'middle_id': 0, 'child_id': 0},
                        {'parent_id': 1, 'middle_id': 1, 'child_id': 0},
                        {'parent_id': 0, 'middle_id': 0, 'child_id': 1},
                        {'parent_id': 1, 'middle_id': 1, 'child_id': 1}])
child.ParentSrc()

In [89]:
(child.ParentSrc() & 'child_id = 0') * child * parent

In [94]:
schema = dj.schema('franklab_nspike', locals())

In [96]:
schema.jobs